In [1]:
import pandas as pd
from os import path
import sqlite3

In [2]:
DATA_DIR = "C:/Users/jake_/Desktop/baseball_etl/data"
HUNDRED_DIR = path.join(DATA_DIR, '100-game-sample')

In [3]:
# create connection
conn = sqlite3.connect(path.join(HUNDRED_DIR, 'baseball.sqlite'))

In [4]:
# load csv data
player_game = pd.read_csv(path.join(HUNDRED_DIR, 'batter-player.csv'))
player = pd.read_csv(path.join(HUNDRED_DIR, 'players.csv'))
game = pd.read_csv(path.join(HUNDRED_DIR, 'games.csv'))
team = pd.read_csv(path.join(DATA_DIR, '2018-season', 'teams.csv'))

In [5]:
# and write it to sql
player_game.to_sql('player_game', conn, index=False, if_exists='replace')
player.to_sql('player', conn, index=False, if_exists='replace')
game.to_sql('game', conn, index=False, if_exists='replace')
team.to_sql('team', conn, index=False, if_exists='replace')

In [6]:
df = pd.read_sql(
    """
    SELECT *
    FROM game
    """, conn)
df.head()

,g_id,home_team,away_team,date,home_final_score,away_final_score,start_time,venue_name,umpire_1B,umpire_2B,umpire_3B,umpire_HP,weather,wind,attendance,elapsed_time,delay,season
0,201800050,CIN,WAS,2018-04-01,5,6,4:10 PM,Great American Ball Park,James Hoye,Jeff Kellogg,Quinn Wolcott,Marvin Hudson,"50 degrees, partly cloudy","8 mph, L to R",10355,167,0,2018
1,201800073,NYA,TBA,2018-04-03,11,4,4:19 PM,Yankee Stadium,Joe West,Doug Eddings,Marty Foster,Mark Ripperger,"40 degrees, overcast","7 mph, In from CF",46776,209,0,2018
2,201800099,COL,ATL,2018-04-06,3,8,3:11 PM,Coors Field,Bill Welke,Tony Randazzo,Lance Barrett,Nic Lentz,"27 degrees, snow","4 mph, R to L",48216,176,61,2018
3,201800115,PHI,MIA,2018-04-07,20,1,6:08 PM,Citizens Bank Park,Carlos Torres,Kerwin Danley,Paul Nauert,Scott Barry,"46 degrees, cloudy","5 mph, In from CF",33660,201,0,2018
4,201800122,SLN,ARI,2018-04-08,1,4,1:15 PM,Busch Stadium,Rob Drake,Mike Muchlinski,Mike Winters,Tim Timmons,"41 degrees, clear","7 mph, R to L",40468,173,0,2018


In [7]:
pd.read_sql(
    """
    SELECT g_id, home_team, away_team, elapsed_time as length
    FROM game
    """, conn).head()

,g_id,home_team,away_team,length
0,201800050,CIN,WAS,167
1,201800073,NYA,TBA,209
2,201800099,COL,ATL,176
3,201800115,PHI,MIA,201
4,201800122,SLN,ARI,173


# Filtering

In [8]:
pd.read_sql("""
            SELECT g_id, home_team, away_team, elapsed_time as length
            FROM game
            WHERE venue_name = 'Oriole Park at Camden Yards'
            """, conn).head()

,g_id,home_team,away_team,length
0,201800177,BAL,TOR,177
1,201800578,BAL,TBA,147
2,201800810,BAL,WAS,194
3,201800825,BAL,WAS,154
4,201801743,BAL,BOS,168


In [9]:
pd.read_sql("""
            SELECT g_id, home_team, away_team, elapsed_time as length
            FROM game
            WHERE venue_name = 'Oriole Park at Camden Yards' AND away_team = 'WAS'
            """,conn).head()

,g_id,home_team,away_team,length
0,201800810,BAL,WAS,194
1,201800825,BAL,WAS,154


In [10]:
pd.read_sql("""
            SELECT g_id, home_team, away_team, elapsed_time as length
            FROM game
            WHERE home_team = 'COL' OR home_team = 'CIN'
            """,conn).head()

,g_id,home_team,away_team,length
0,201800050,CIN,WAS,167
1,201800099,COL,ATL,176
2,201800755,COL,CIN,180
3,201800940,CIN,SLN,231
4,201801077,CIN,DET,212


In [11]:
# the last query could also be written as:
pd.read_sql("""
            SELECT g_id, home_team, away_team, elapsed_time as length
            FROM game
            WHERE home_team IN ('COL', 'CIN')
            """,conn).head()

,g_id,home_team,away_team,length
0,201800050,CIN,WAS,167
1,201800099,COL,ATL,176
2,201800755,COL,CIN,180
3,201800940,CIN,SLN,231
4,201801077,CIN,DET,212


In [12]:
# SQL also used with negation 
pd.read_sql("""
            SELECT g_id, home_team, away_team, elapsed_time as length
            FROM game
            WHERE home_team NOT IN ('COL', 'CIN')
            """,conn).head()

,g_id,home_team,away_team,length
0,201800073,NYA,TBA,209
1,201800115,PHI,MIA,201
2,201800122,SLN,ARI,173
3,201800128,CLE,KCA,175
4,201800133,HOU,SDN,145


# Joining, or Selecting From Multiple Tables

In [13]:
# table.column_name syntax
pd.read_sql("""
            SELECT 
                player.first_name,
                player.last_name,
                player.team,
                team.lgID,
                team.divID
            FROM player, team
            """,conn)

,first_name,last_name,team,lgID,divID
0,Jon,Lester,CHN,NL,W
1,Jon,Lester,CHN,NL,E
2,Jon,Lester,CHN,AL,E
3,Jon,Lester,CHN,AL,E
4,Jon,Lester,CHN,AL,C
...,...,...,...,...,...
27325,Caleb,Frare,CHA,NL,C
27326,Caleb,Frare,CHA,AL,E
27327,Caleb,Frare,CHA,AL,W
27328,Caleb,Frare,CHA,AL,E


There is something weird going on here. The problem is we haven't told SQL how the 'player' and 'team' tables are related. When you  don't include that info, SQL doesn't try to figure it out or complain. Instead it returns a crossjoin, i.e. EVERY ROW in the 'player' table gets matched up with EVERY ROW in the 'team' table 

In [14]:
crossjoin = pd.read_sql("""
            SELECT 
                player.first_name,
                player.last_name,
                player.team as player_team,
                team.teamID as team_team,
                team.lgID,
                team.divID
            FROM player, team
            """,conn)
crossjoin

,first_name,last_name,player_team,team_team,lgID,divID
0,Jon,Lester,CHN,ARI,NL,W
1,Jon,Lester,CHN,ATL,NL,E
2,Jon,Lester,CHN,BAL,AL,E
3,Jon,Lester,CHN,BOS,AL,E
4,Jon,Lester,CHN,CHA,AL,C
...,...,...,...,...,...,...
27325,Caleb,Frare,CHA,SLN,NL,C
27326,Caleb,Frare,CHA,TBA,AL,E
27327,Caleb,Frare,CHA,TEX,AL,W
27328,Caleb,Frare,CHA,TOR,AL,E


This makes it clear it's a crossjoin -- every line is getting linked up with every team in the team table. Also, since we have a 911 row player table and a 30 row team table, that means we should get back 911*30=27330 rows

In [15]:
# Sure enough
crossjoin.shape

(27330, 6)

In [16]:
pd.read_sql("""
            SELECT 
                player.first_name,
                player.last_name,
                player.team,
                team.lgID,
                team.divID
            FROM player, team
            WHERE team.teamID = player.team
            """,conn)

,first_name,last_name,team,lgID,divID
0,Jon,Lester,CHN,NL,C
1,Cole,Hamels,CHN,NL,C
2,Clayton,Kershaw,LAN,NL,W
3,Alex,Wood,LAN,NL,W
4,Wei-Yin,Chen,MIA,NL,E
...,...,...,...,...,...
873,Jake,Newberry,KCA,AL,C
874,Ryan,Burr,CHA,AL,C
875,Taylor,Guerrieri,TOR,AL,E
876,Jose Manuel,Fernandez,TOR,AL,E


In [17]:
# Again adding the 'team' column from table makes it more clear:
pd.read_sql("""
            SELECT 
                player.first_name,
                player.last_name,
                player.team as player_team,
                team.teamID as team_team,
                team.lgID,
                team.divID
            FROM player, team
            WHERE team.teamID = player.team
            """,conn)

,first_name,last_name,player_team,team_team,lgID,divID
0,Jon,Lester,CHN,CHN,NL,C
1,Cole,Hamels,CHN,CHN,NL,C
2,Clayton,Kershaw,LAN,LAN,NL,W
3,Alex,Wood,LAN,LAN,NL,W
4,Wei-Yin,Chen,MIA,MIA,NL,E
...,...,...,...,...,...,...
873,Jake,Newberry,KCA,KCA,AL,C
874,Ryan,Burr,CHA,CHA,AL,C
875,Taylor,Guerrieri,TOR,TOR,AL,E
876,Jose Manuel,Fernandez,TOR,TOR,AL,E


In [18]:
pd.read_sql("""
            SELECT 
               player_game.g_id,
               player.first_name,
               player.last_name,
               player.team,
               team.lgID,
               team.divID,
               player_game.n_atbats,
               player_game.hit,
               player_game.homerun
            FROM player, team, player_game
            WHERE 
                team.teamID = player.team AND
                player_game.batter_id = player.id
            """,conn)

,g_id,first_name,last_name,team,lgID,divID,n_atbats,hit,homerun
0,201800798,Jon,Lester,CHN,NL,C,3,2.0,0.0
1,201801966,Jon,Lester,CHN,NL,C,2,1.0,0.0
2,201801978,Cole,Hamels,CHN,NL,C,1,0.0,0.0
3,201801628,Clayton,Kershaw,LAN,NL,W,3,1.0,0.0
4,201802188,Clayton,Kershaw,LAN,NL,W,3,1.0,0.0
...,...,...,...,...,...,...,...,...,...
2018,201802105,Rowdy,Tellez,TOR,AL,E,5,2.0,0.0
2019,201802250,Rowdy,Tellez,TOR,AL,E,3,1.0,0.0
2020,201802129,Abiatal,Avelino,SFN,NL,W,2,0.0,0.0
2021,201802421,Abiatal,Avelino,SFN,NL,W,2,0.0,0.0


In [19]:
pd.read_sql("""
            SELECT 
                p.first_name,
                p.last_name,
                p.team,
                t.lgID,
                t.divID,
                pg.*
            FROM player AS p, team AS t, player_game AS pg
            WHERE
                t.teamID = p.team AND
                pg.batter_id = p.id
            """,conn)

,first_name,last_name,team,lgID,divID,batter_id,g_id,batter,team,opp,rbi,strikeout,out,n_atbats,homerun,hit
0,Jon,Lester,CHN,NL,C,452657,201800798,J.Lester,CHN,PIT,1,0.0,1.0,3,0.0,2.0
1,Jon,Lester,CHN,NL,C,452657,201801966,J.Lester,CHN,NYN,2,1.0,1.0,2,0.0,1.0
2,Cole,Hamels,CHN,NL,C,430935,201801978,C.Hamels,CHN,NYN,0,1.0,1.0,1,0.0,0.0
3,Clayton,Kershaw,LAN,NL,W,477132,201801628,C.Kershaw,LAN,MIL,0,1.0,2.0,3,0.0,1.0
4,Clayton,Kershaw,LAN,NL,W,477132,201802188,C.Kershaw,LAN,SLN,1,0.0,2.0,3,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,Rowdy,Tellez,TOR,AL,E,642133,201802105,R.Tellez,TOR,CLE,1,3.0,3.0,5,0.0,2.0
2019,Rowdy,Tellez,TOR,AL,E,642133,201802250,R.Tellez,TOR,BAL,1,2.0,2.0,3,0.0,1.0
2020,Abiatal,Avelino,SFN,NL,W,623214,201802129,A.Avelino,SFN,MIL,0,2.0,2.0,2,0.0,0.0
2021,Abiatal,Avelino,SFN,NL,W,623214,201802421,A.Avelino,SFN,LAN,0,1.0,2.0,2,0.0,0.0


# Combining Join and Other Filters

In [20]:
pd.read_sql("""
            SELECT 
                p.first_name,
                p.last_name,
                p.team,
                t.lgID,
                t.divID,
                pg.*
            FROM player AS p, team AS t, player_game AS pg
            WHERE
                t.teamID = p.team AND
                pg.batter_id = p.id AND
                p.team = 'NYN'
            """,conn)

,first_name,last_name,team,lgID,divID,batter_id,g_id,batter,team,opp,rbi,strikeout,out,n_atbats,homerun,hit
0,Jacob,deGrom,NYN,NL,E,594798,201801002,J.deGrom,NYN,ATL,0,0.0,2.0,2,0.0,0.0
1,Jacob,deGrom,NYN,NL,E,594798,201801382,J.deGrom,NYN,PHI,0,0.0,2.0,2,0.0,0.0
2,Jacob,deGrom,NYN,NL,E,594798,201801978,J.deGrom,NYN,CHN,1,0.0,2.0,4,0.0,2.0
3,Noah,Syndergaard,NYN,NL,E,592789,201801966,N.Syndergaard,NYN,CHN,0,0.0,1.0,2,0.0,0.0
4,Noah,Syndergaard,NYN,NL,E,592789,201802422,N.Syndergaard,NYN,MIA,0,2.0,2.0,3,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Luis,Guillorme,NYN,NL,E,641645,201801002,L.Guillorme,NYN,ATL,0,0.0,2.0,2,0.0,0.0
60,Jeff,McNeil,NYN,NL,E,643446,201801806,J.McNeil,NYN,BAL,1,0.0,4.0,6,0.0,1.0
61,Jeff,McNeil,NYN,NL,E,643446,201801966,J.McNeil,NYN,CHN,0,0.0,1.0,1,0.0,0.0
62,Jeff,McNeil,NYN,NL,E,643446,201801978,J.McNeil,NYN,CHN,0,0.0,3.0,5,0.0,1.0


### LIMIT/TOP

In [21]:
# Sometimes you want to make sure a query works and see what cols yoiu get back before you run the whole thing
pd.read_sql("""
            SELECT * 
            FROM player
            LIMIT 5
            """, conn)

,id,first_name,last_name,team,position
0,452657,Jon,Lester,CHN,pitcher
1,430935,Cole,Hamels,CHN,pitcher
2,477132,Clayton,Kershaw,LAN,pitcher
3,622072,Alex,Wood,LAN,pitcher
4,612672,Wei-Yin,Chen,MIA,pitcher


In [22]:
# Syntax for Microsoft SQL Server

# pd.read_sql("""
#             SELECT TOP 5 * 
#             FROM player
#             """, conn)

### DISTINCT

In [23]:
# Including DISTINCT right after SELECT drops duplicate observations

pd.read_sql("""
            SELECT DISTINCT umpire_1b, umpire_2B, umpire_3B, umpire_HP
            FROM game
            """, conn)

,umpire_1B,umpire_2B,umpire_3B,umpire_HP
0,James Hoye,Jeff Kellogg,Quinn Wolcott,Marvin Hudson
1,Joe West,Doug Eddings,Marty Foster,Mark Ripperger
2,Bill Welke,Tony Randazzo,Lance Barrett,Nic Lentz
3,Carlos Torres,Kerwin Danley,Paul Nauert,Scott Barry
4,Rob Drake,Mike Muchlinski,Mike Winters,Tim Timmons
...,...,...,...,...
93,Larry Vanover,Hunter Wendelstedt,Chris Guccione,David Rackley
94,Jim Reynolds,Mark Wegner,John Tumpane,John Libka
95,Bruce Dreckman,Chad Fairchild,Kerwin Danley,Mike Estabrook
96,Pat Hoberg,Ed Hickox,Gerry Davis,Todd Tichenor


### UNION

In [25]:
# This query gives us number of at bats, hits, and homeruns 
# for every player and game in our sample, whether the player actually played

full_player_by_week_table = pd.read_sql("""
                                        SELECT a.date, a.team, a.opp, a.venue_name, a.first_name, a.last_name,
                                                b.n_atbats, b.hit, b.homerun
                                        FROM 
                                            (SELECT g_id, date, home_team as team, away_team as opp, id,
                                                    venue_name, first_name, last_name
                                             FROM game, player
                                             WHERE 
                                                 game.home_team = player.team AND
                                                 player.position = 'fielder'
                                             UNION
                                             SELECT g_id, date, home_team as team, away_team as opp, id,
                                                    venue_name, first_name, last_name
                                             FROM game, player
                                             WHERE
                                                 game.away_team = player.team AND
                                                 player.position = 'fielder') AS a
                                        LEFT JOIN player_game AS b ON a.g_id = b.g_id AND a.id = b.batter_id
                                        """, conn)

full_player_by_week_table

,date,team,opp,venue_name,first_name,last_name,n_atbats,hit,homerun
0,2018-04-01,CIN,WAS,Great American Ball Park,Matt,Wieters,NaN,NaN,NaN
1,2018-04-01,CIN,WAS,Great American Ball Park,Mark,Reynolds,NaN,NaN,NaN
2,2018-04-01,CIN,WAS,Great American Ball Park,Joey,Votto,4.0,1.0,0.0
3,2018-04-01,CIN,WAS,Great American Ball Park,Miguel,Montero,4.0,0.0,0.0
4,2018-04-01,CIN,WAS,Great American Ball Park,Ryan,Zimmerman,5.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
3226,2018-09-30,NYN,MIA,Citi Field,Dominic,Smith,3.0,0.0,0.0
3227,2018-09-30,NYN,MIA,Citi Field,Magneuris,Sierra,3.0,1.0,0.0
3228,2018-09-30,NYN,MIA,Citi Field,Amed,Rosario,3.0,0.0,0.0
3229,2018-09-30,NYN,MIA,Citi Field,Jeff,McNeil,4.0,2.0,0.0


In [26]:
full_player_by_week_table.query("last_name == 'Acuna'")

,date,team,opp,venue_name,first_name,last_name,n_atbats,hit,homerun
99,2018-04-06,COL,ATL,Coors Field,Ronald,Acuna,NaN,NaN,NaN
1026,2018-06-05,SDN,ATL,Petco Park,Ronald,Acuna,NaN,NaN,NaN
1126,2018-06-13,ATL,NYN,SunTrust Park,Ronald,Acuna,NaN,NaN,NaN
2023,2018-08-08,WAS,ATL,Nationals Park,Ronald,Acuna,5.0,2.0,1.0
2166,2018-08-14,ATL,MIA,SunTrust Park,Ronald,Acuna,5.0,3.0,2.0
2954,2018-09-19,ATL,SLN,SunTrust Park,Ronald,Acuna,4.0,0.0,0.0
3018,2018-09-23,ATL,PHI,SunTrust Park,Ronald,Acuna,1.0,0.0,0.0
